In [52]:

from Simulation_code.Simulation import *
from lib_code.NMF_lib import get_house_keeping_genes
from pipeline_code.setup_data_run import split_train_test, get_split_data


# setup

In [53]:
cohort_name = 'CRC+SCLC'
N = 185
N_test = 120

# paper params
# ts = [1,2,3,4,5]
Ks = np.arange(1,11)

# demo params
ts = [1]
Ks = np.arange(1,11)

data_path = root_path/f'data/{cohort_name}/unsupervised1'
em_runs_output_path = root_path/f'results/{cohort_name}/unsupervised1'
get_test = True

mkdir(em_runs_output_path, recursive=True)
plots_path = root_path/f'plots/{cohort_name}/unsupervised1'
mkdir(plots_path, recursive=True)

em_runs_output_path

PosixPath('/Users/elafallik/Documents/submission-github/results/CRC+SCLC/unsupervised1')

In [54]:
ref_seq = pd.read_csv(root_path/'data/ref-seq.txt', dtype='object', header=None).values[:, 0]

# get data

In [55]:
data = pd.read_csv(data_path.parent/'counts-filtered.csv', index_col=0)
bg = pd.read_csv(data_path.parent/'background-filtered.csv', index_col=0)

bg.to_csv(data_path/'background-filtered.csv')
data.shape, data.values.max()

((18182, 305), 7176)


## get samples

In [56]:
cohort1_name = 'SCLC'
cohort2_name = 'CRC'

In [57]:
cohort1_samples = np.array([s for s in data.columns if s.startswith('SCLC')])
cohort2_samples = np.array([s for s in data.columns if s.startswith('CRC')])
healthy_samples = np.array([s for s in data.columns if s.startswith('PS')])
samples = data.columns

get_test = True
print(cohort1_samples.shape, cohort2_samples.shape, healthy_samples.shape, samples.shape, data.shape, data.values.max())

(139,) (86,) (80,) (305,) (18182, 305) 7176


## get split train test

In [58]:

M = len(ref_seq)
samples_dir_readfrom = data_path.parent
data_dir_saveto = data_path
data_name = ''
dataset = ''
split_train_test(N, M, N_test, samples_dir_readfrom, data_dir_saveto, data_name, dataset,
                     sample_split_dict=None, score_genes=False, take_first=True, prefix='', get_test=get_test)
train_data, test_data = get_split_data(data_dir_saveto, data_name, get_test=get_test)

train_samples = train_data.columns
samples = train_samples
if get_test:
    test_samples = test_data.columns
    samples = np.concatenate([train_samples, test_samples])
    print(train_samples.shape, test_samples.shape)
else: print(train_samples.shape)
samples

(185,) (120,)


array(['CRC0001-2752_NA_H3K4me3-L_03092018-L',
       'SCLC0147-5_NA_H3K4me3-38_26032021-13',
       'SCLC0063-699_NA_H3K4me3-38_26032021-13',
       'SCLC0196-711_NA_H3K4me3-38_26032021-13',
       'SCLC0261-450_NA_H3K4me3-38_26032021-13',
       'CRC0053-3882_NA_H3K4me3-L_19022020-L',
       'SCLC0039-499_NA_H3K4me3-39_26032021-13',
       'CRC0009-1996_NA_H3K4me3-L_17122019-L',
       'SCLC0180-112_NA_H3K4me3-39_26032021-13',
       'SCLC0059-723_NA_H3K4me3-39_26032021-13',
       'SCLC0006-464_NA_H3K4me3-39_26032021-13',
       'CRC0041-3870_NA_H3K4me3-L_17022020-L',
       'SCLC0176-178_NA_H3K4me3-39_26032021-13',
       'SCLC0041-509_NA_H3K4me3-38_26032021-13',
       'SCLC0049-585_NA_H3K4me3-39_26032021-13',
       'SCLC0050-605_NA_H3K4me3-38_26032021-13',
       'SCLC0031-445_NA_H3K4me3-39_26032021-13',
       'CRC0010-1913_NA_H3K4me3-L_17122019-L',
       'SCLC0025-457_NA_H3K4me3-38_26032021-13',
       'SCLC0035-586_NA_H3K4me3-39_26032021-13',
       'PS0046-3_EDTA-FreshChIP_

In [59]:

healthy_samples_train = np.intersect1d(healthy_samples, train_samples)
cohort1_samples_train = np.intersect1d(cohort1_samples, train_samples)
cohort2_samples_train = np.intersect1d(cohort2_samples, train_samples)
healthy_samples_train_idx = [np.argwhere(train_samples == s)[0, 0] for s in healthy_samples_train]
cohort1_samples_train_idx = [np.argwhere(train_samples == s)[0, 0] for s in cohort1_samples_train]
cohort2_samples_train_idx = [np.argwhere(train_samples == s)[0, 0] for s in cohort2_samples_train]
train_samples_idx = [np.argwhere(samples == s)[0, 0] for s in train_samples]
short_names_train = np.array([s.split('_')[0] for s in train_samples])

if get_test:
    healthy_samples_test = np.intersect1d(healthy_samples, test_samples)
    cohort1_samples_test = np.intersect1d(cohort1_samples, test_samples)
    cohort2_samples_test = np.intersect1d(cohort2_samples, test_samples)
    healthy_samples = np.concatenate([healthy_samples_train, healthy_samples_test])
    cohort1_samples = np.concatenate([cohort1_samples_train, cohort1_samples_test])
    cohort2_samples = np.concatenate([cohort2_samples_train, cohort2_samples_test])

    healthy_samples_test_idx = [np.argwhere(test_samples == s)[0, 0] for s in healthy_samples_test]
    cohort1_samples_test_idx = [np.argwhere(test_samples == s)[0, 0] for s in cohort1_samples_test]
    cohort2_samples_test_idx = [np.argwhere(test_samples == s)[0, 0] for s in cohort2_samples_test]
    test_samples_idx = [np.argwhere(samples == s)[0, 0] for s in test_samples]
    short_names_test = np.array([s.split('_')[0] for s in test_samples])
    print(healthy_samples_train.shape, cohort1_samples_train.shape, cohort2_samples_train.shape)
    print(healthy_samples_test.shape, cohort1_samples_test.shape, cohort2_samples_test.shape)
else:
    healthy_samples = healthy_samples_train
    cohort1_samples = cohort1_samples_train
    cohort2_samples = cohort2_samples_train
    print(healthy_samples_train.shape, cohort1_samples_train.shape, cohort2_samples_train.shape)

(47,) (81,) (57,)
(33,) (58,) (29,)


In [60]:
if not (data_path/f'Healthy+{cohort_name}-samples.csv').exists():
    pd.DataFrame(samples).to_csv(data_path/f'Healthy+{cohort_name}-samples.csv')
    np.savetxt(data_path/f'Healthy+{cohort_name}-samples.txt', samples, fmt='%s')

    pd.DataFrame(healthy_samples).to_csv(data_path/'Healthy-samples.csv')
    np.savetxt(data_path/'Healthy-samples.txt', healthy_samples, fmt='%s')

    cohort_samples = np.concatenate([cohort1_samples_train, cohort2_samples_train, cohort1_samples_test, cohort2_samples_test])
    pd.DataFrame(cohort_samples).to_csv(data_path/f'{cohort_name}-samples.csv')
    np.savetxt(data_path/f'{cohort_name}-samples.txt', cohort_samples, fmt='%s')


In [61]:
train_data.values.max(), test_data.values.max(), train_data.shape

(4146, 7176, (18182, 185))

# filter genes

In [62]:

gene_list = pd.read_csv(data_path.parent/'Genes.select.csv').values[:, 0]
mkdir(data_path/'selected-genes')

if not (data_path/'selected-genes/train.csv').exists():
    train_data.loc[gene_list].to_csv(data_path/'selected-genes/train.csv')
    test_data.loc[gene_list].to_csv(data_path/'selected-genes/test.csv')

gene_list

array(['A1BG', 'A1CF', 'A2M', ..., 'ZSWIM6', 'ZW10', 'ZYG11A'],
      dtype=object)

In [63]:
ref_seq = pd.read_csv(data_path.parent/'Genes.select.csv').values[:, 0]
train_data = train_data.loc[ref_seq]
test_data = test_data.loc[ref_seq]
M = len(ref_seq)
M

7000

In [64]:

house_keeping = np.array([g for g in get_house_keeping_genes(gen_path=root_path) if g in ref_seq])
house_keeping_idx = np.array([i for i in range(len(ref_seq)) if ref_seq[i] in house_keeping])

len(house_keeping_idx)

1365